In [ ]:
# --- Step 1: Install necessary libraries ---
# Run these commands in separate Colab cells or at the beginning of your script.
!pip install ultralytics
!pip install roboflow
# !pip install PyYAML # Often a dependency for data.yaml processing, though usually handled by ultralytics

# --- Step 2: Mount Google Drive (Optional but Recommended for saving models) ---
# If you want to save your trained model (best.pt) directly to your Google Drive,
# uncomment and run the following lines.
# from google.colab import drive
# drive.mount('/content/drive')
# print("Google Drive mounted successfully!")

# --- Step 3: Download your dataset from Roboflow ---
# Replace 'CTTn5Sh5JaQUn0FdOSHU' with your actual Roboflow API Key if it's different.
# This will download your dataset into a folder in your Colab environment.
# The folder name will typically be 'crime-scene-oma5u-7' as per your project details.

from roboflow import Roboflow
import os

print("Downloading dataset from Roboflow...")
try:
    rf = Roboflow(api_key="CTTn5Sh5JaQUn0FdOSHU") # Your Roboflow API Key
    project = rf.workspace("crimesceneobjectsdetection").project("crime-scene-oma5u")
    version = project.version(7)
    dataset = version.download("yolov8") # This downloads the dataset and creates data.yaml
    !pip install roboflow



    # The dataset.location will give you the path to the downloaded dataset folder
    data_yaml_path = os.path.join(dataset.location, "data.yaml")
    print(f"Dataset downloaded to: {dataset.location}")
    print(f"data.yaml path: {data_yaml_path}")

except Exception as e:
    print(f"Error downloading dataset from Roboflow: {e}")
    print("Please check your API key, workspace, project, and version details.")
    exit()

# --- Step 4: Initialize and Train the YOLOv8 Model ---
# This is the core training step.

from ultralytics import YOLO

print("\n--- Starting YOLOv8 Model Training ---")

try:
    # Initialize a new YOLOv8s model.
    # You can start with 'yolov8s.pt' (small model) or 'yolov8m.pt' (medium) etc.,
    # depending on your computational resources and desired model size.
    # If you have a previously trained model and want to resume training,
    # you can load it like: model = YOLO('path/to/your/last.pt')
    model = YOLO('yolov8s.pt')

    # Train the model
    # data: path to the dataset configuration file (data.yaml)
    # epochs: number of training epochs (start with a small number, e.g., 50, and increase if needed)
    # imgsz: input image size (e.g., 640 for 640x640 images)
    # batch: batch size (adjust based on Colab GPU memory, common values: 16, 32, 64)
    # name: a name for your training run, results will be saved in runs/detect/your_name
    # device: '0' for GPU, 'cpu' for CPU. Colab usually has a GPU at '0'.
    results = model.train(
        data=data_yaml_path,
        epochs=50, # You can adjust this number
        imgsz=640, # You can adjust this number
        batch=16,  # Adjust based on your Colab GPU memory
        name='crime_detection_training', # Name for your training run
        device='0' # Use GPU (Colab usually provides one)
    )

    print("\n--- Model training completed! ---")
    print("Your training results, including the 'best.pt' and 'last.pt' model weights,")
    print("will be saved in the 'runs/detect/crime_detection_training' directory.")

    # You can find your best.pt file here:
    # For example: /content/runs/detect/crime_detection_training/weights/best.pt

    # If you mounted Google Drive, you can copy the best.pt file to your Drive:
    # import shutil
    # source_path = '/content/runs/detect/crime_detection_training/weights/best.pt'
    # destination_path = '/content/drive/MyDrive/yolov8_crime_detection_best.pt' # Change this path as desired
    # shutil.copy(source_path, destination_path)
    # print(f"Copied best.pt to Google Drive: {destination_path}")

except Exception as e:
    print(f"An error occurred during model training: {e}")
    print("Please check your dataset path, training parameters, and Colab environment.")